In [ ]:
%%time
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()
tf.set_random_seed(777)
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

learning_rate = .0005
training_epochs = 20
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# Level 1
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # padding 있으나 마나
print(L1.shape)

# Level 2
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])
print(L2.shape)

# Level 3
W = tf.Variable(tf.random_normal([3, 3, 64, 256], stddev=.01))
L3 = tf.nn.conv2d(L2, W, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L3.shape)
# L3_flat = tf.reshape(L3, [-1, 7 * 7 * 256])

# Level 3
W0 = tf.Variable(tf.random_normal([3, 3, 256, 128], stddev=.01))
L4 = tf.nn.conv2d(L3, W0, strides=[1,1,1,1], padding='SAME')
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L4.shape)
L4_flat = tf.reshape(L4, [-1, 2 * 2 * 128])

# Final FC 7X7X64 inputs -> 10 outpues
W3 = tf.get_variable('W3', shape=[2 * 2 * 128, 256],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([256]))

logits = tf.matmul(L4_flat, W3) + b

W4 = tf.get_variable('W4', shape=[256, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(logits, W4) + b1

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X : batch_xs, Y : batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    print('Epoch:{:04d}'.format(epoch + 1), 'cost = {:.9f}'.format(avg_cost))
print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
    X : mnist.test.images, Y : mnist.test.labels
}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print('Label:', sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print('Prediction:', sess.run(
    tf.argmax(logits, 1), feed_dict={X:mnist.test.images[r:r+1]}))